<a href="https://colab.research.google.com/github/NicKylis/letter_recognition/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import tensorflow as tf
print ("hello world!")

hello world!


In [2]:
print('Hello why google colabs works as good as me when thlep?')

Hello why google colabs works as good as me when thlep?
